# Hacker News:  Data Análisis.
[Hacker news](https://news.ycombinator.com) es un sitio de noticias tecnológicas extremadamente popular entre los circulos de los startup y filio-tecnólogos. Los usuarios postean publicaciones de noticias relacionadas, y mediante el sistema de votos, se destacan estas dentro de la página principal. Dentro de la página están los post clasificados como [Ask HN](https://news.ycombinator.com/ask), que son las preguntas realizadas a la comunidad de Hacker News y los post llamados [Show HN](https://news.ycombinator.com/show) dedicado a mostrar los proyectos, productos, etc, que realiza la comunidad.  

El objetivo de este proyecto es comparar los tipos de post anteriores y ser capaces de responder las siguientes preguntas:

* ¿Qué tipo de post, en promedio, recibe más comentarios?
* ¿ Existe una relación entre la hora en que se publica un post, y el número de comentarios que este recibe?


## Los datos

El set de datos se componen de los link a noticias más populares, el puntaje que obtiene cada posteo y el numero de comentarios que recibe cada uno. 

Es posible obtener el set de datos en el siguiente [enlace](https://www.kaggle.com/hacker-news/hacker-news-posts) y para este trabajo se extrajo una muestra de 20.000 filas de un total de 300.000.Una descripción de cada encabezado lo puedes encontrar en el mismo enlace anterior.

Se comienza abriendo el set de datos desde el archivo hacker_news.csv  y guardandolo en una lista de lista llamado hn. 


In [15]:
opened_file = open('hacker_news.csv')
from csv import reader
hn = list(reader(opened_file))
print(hn[0:6])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


Extraemos el encabezado a una variable llamada headers, removiendo esta linea del set de datos.

In [19]:
header = hn[0]
hn = hn[1:]
print(header)
print('\n')
print(hn[0:6])

['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


[['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12'], ['10482257', 'Title II kills investment? Comcast and other ISPs are now spending more', 'http://arstechnica.com/business/2015/10/comcast-and-other-isps-boost-network-investment-despite-net-neutrality/', '53', '22', 'Deino

Como se indico anteriormente, se esta interesado solamente en los post cuyos títulos comiencen con Ask HN o Show HN (en la columna title), por ello se crea una nueva lista de lista conteniendo solamente estos datos.  

In [32]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title_lw = title.lower() #convertimos todo a minúsculas para no tener problemas
    if title_lw.startswith('ask hn'):
        ask_posts.append(row)
    elif title_lw.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

print("Total de 'Ask HN'  posts:",len(ask_posts))
print("Total de 'Show HN' posts:",len(show_posts))
print("Total de 'others'  posts:",len(other_posts))
        
        

Total de 'Ask HN'  posts: 1744
Total de 'Show HN' posts: 1162
Total de 'others'  posts: 17192


Ahora averiguaremos cual de los post en promedio recibe mas comentarios.

In [37]:
total_ask_comments = 0
for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
    
avg_ask_comments = total_ask_comments / len(ask_posts)
print("Promedio de comentarios en Ask HN:", avg_ask_comments)


total_show_comments = 0
for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
    
avg_show_comments = total_show_comments / len(show_posts)
print("Promedio de comentarios en Show HN:", avg_show_comments)

Promedio de comentarios en Ask HN: 14.038417431192661
Promedio de comentarios en Show HN: 10.31669535283993


* Primer Hallazgo : En promedio los posts realizados en Ask HN reciben 4 comentarios más que en Show HN. Debido que este tipo de posts tiende a recibir mayor numero de comentarios, el posterior análisis se realizara únicamente sobre ellos.

 Como se indico al comienzo, nuestro segundo objetivo es determinar si a una determinada hora se producen mayor número de comentarios. Esto último puede resultar de importancia comercial. 
 
 A continuación se calculara la cantidad de posts creados por cada hora del día junto con el número de comentarios recibidos en esa hora. 
 
 Recordemos que la última columna nos entrega la fecha y ahora en el cual el post fue creado, con el formato siguiente: created_at (M/D/Y H:M), ejemplo de contenido en result_list: 
 
 [['8/16/2016 9:55', 2], ['11/22/2015 13:43', 28], ['5/2/2016 10:14', 1], ['8/2/2016 14:20', 1]]

In [57]:
import datetime as dt  # se utiliza el modulo datetime para hacer parse
result_list = []       # para formar una lista de lista 

for row in ask_posts:
    created_at = row[6]
    num_comments = int(row[3])
    result_list.append([created_at, num_comments])

counts_by_hour = {}  #cuantas veces se repite una hora
comments_by_hour = {} # en una hora en particular cuantos comentarios hubieron

for row in result_list:  # se hace parse de la la fecha y hora en que se hizo el comentario
    string_date = row[0]
    ob_created_at = dt.datetime.strptime(string_date,"%m/%d/%Y %H:%M")  
    hour_string = dt.datetime.strftime(ob_created_at,"%H:%M" )
    
    if hour_string not in counts_by_hour:
        counts_by_hour[hour_string] = 1
        comments_by_hour[hour_string] = row[1]
        
    elif hour_string in counts_by_hour:
        counts_by_hour[hour_string] += 1
        comments_by_hour[hour_string] += row[1]

Ahora con los diccionarios realizados, se calculara el promedio de comentarios hechos a cada post segun una determinada hora

In [77]:
avg_by_hour = []
for key in counts_by_hour:
    num_comments = int(comments_by_hour[key])
    num_counts = int(counts_by_hour[key])
    avg_by_hour.append([key, num_comments / num_counts])

Ahora de la lista de lista final avg_by_hour, se mostrara las cinco primeras filas con mayores valores:

[[607.5, '16:43'], [450.0, '13:04'],..]

In [86]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1], row[0]]) # se hace swap para poder utilizar sorted()

sorted_swap = sorted(swap_avg_by_hour, reverse=True)

print("Top 5 Hours for Ask Posts Comments")
for row in sorted_swap[:6]:
    ob_hora = dt.datetime.strptime(row[1],"%H:%M")
    string_hora = dt.datetime.strftime(ob_hora,"%H:%M")
    print("A la hora {} tiene {} comentarios por post".format(string_hora,row[0]))

Top 5 Hours for Ask Posts Comments
A la hora 16:43 tiene 607.5 comentarios por post
A la hora 13:04 tiene 450.0 comentarios por post
A la hora 11:52 tiene 442.0 comentarios por post
A la hora 17:02 tiene 401.0 comentarios por post
A la hora 10:25 tiene 295.0 comentarios por post
A la hora 17:18 tiene 259.5 comentarios por post


## Conclusiones
Hacker News es uno de los sitios más populares entre los círculos de start-up y amantes de la tecnología. Por ello es importante saber publicar tus artículos relacionados a fin de tener el mayor impacto y divulgación posible.

En este proyecto se han encontrado dos mayores conclusiones a saber:

1.- Conviene realizar tus publicaciones con la etiqueta de Ask Posts, puesto que reciben mayor numero de comentarios.
2.- La mejor hora para publicar es aproximadamente a las 16:43 horas, seguido de las horas de almuerzo 13:00 o 11:50.

Siguiendo estas dos simples reglas, te aseguras que el esfuerzo de tu publicación no sea invisible, o que almeno reciba una justa atención.